In [34]:
# import modules

import dual_autodiff as df
import numpy as np
import matplotlib.pyplot as plt

In [35]:
# initialise dual number
x = df.Dual(2, 1)
print(f"x.real = {x.real}, x.dual = {x.dual}")

x.real = 2, x.dual = 1


In [22]:
# printing dual number
print(x)

Dual(real=2, dual=1)


In [23]:
# basic arithmetic operations

x = df.Dual(2, 1)
y = df.Dual(3, 2)

print(f"x + y = {x + y}")
print(f"x - y = {x - y}")
print(f"x * y = {x * y}")
print(f"x / y = {x / y}")

x + y = Dual(real=5, dual=3)
x - y = Dual(real=-1, dual=-1)
x * y = Dual(real=6, dual=7)
x / y = Dual(real=0.6666666666666666, dual=-0.25)


In [24]:
# trigonometric operations

print(f"sin(x) = {x.sin()}")
print(f"cos(x) = {x.cos()}")
print(f"tan(x) = {x.tan()}")

sin(x) = Dual(real=0.9092974268256817, dual=-0.4161468365471424)
cos(x) = Dual(real=-0.4161468365471424, dual=-0.9092974268256817)
tan(x) = Dual(real=-2.185039863261519, dual=5.774399204041917)


In [25]:
# the natural logarithm and exponential

print(f"log(x) = {x.log()}")
print(f"exp(x) = {x.exp()}")

log(x) = Dual(real=0.6931471805599453, dual=0.5)
exp(x) = Dual(real=7.38905609893065, dual=7.38905609893065)


In [26]:
# powers

print(f"x**y = {x**y}")

x**y = Dual(real=8, dual=35.090354888959126)


In [27]:
# automatic differentiation

x = df.Dual(1.5, 1) #initialise x = 1.5, with x.dual = 1 to allow for differentiation

function = x.sin().log() + x**2 * x.cos() #f(x) = log(sin(x)) + x^2 * cos(x)

print(f"log(sin(x)) + x^2 cos(x) = {function.real}")
print(f"d/dx(log(sin(x)) + x^2 cos(x)) = {function.dual}")

log(sin(x)) + x^2 cos(x) = 0.15665054756073515
d/dx(log(sin(x)) + x^2 cos(x)) = -1.9612372705533612


In [28]:
# comparison to analytic and numerical methods:

def f(x):
    return np.log(np.sin(x)) + x**2 * np.cos(x)

def df(x):
    return np.cos(x) / np.sin(x) + 2*x*np.cos(x) - x**2 * np.sin(x)

print(f"analytic df/dx = {df(x=1.5)}")

def central(f, x, h=1e-5):
    return (f(x+h) - f(x-h))/(2*h)

print(f"numerical df/dx = {central(f, x=1.5)}")

analytic df/dx = -1.9612372705533612
numerical df/dx = -1.9612372706409584


In [8]:
import dual_autodiff_x.dual_autodiff_x as dfx
# from dual_autodiff_x.dual_autodiff_x import Dual
import time

In [30]:
def timer(func, *args):
    start = time.time()
    func(*args)
    end = time.time()
    return end - start

In [40]:
def df_tester():
    x = df.Dual(2, 1)
    y = df.Dual(3, 2)

    z1 = x + y
    z2 = x - y
    z3 = x * y
    z4 = x / y
    
    z5 = x.sin()
    z6 = x.cos()
    z7 = x.tan()
    z8 = x.log()
    z9 = x.exp()

    z10 = x**y

    a = df.Dual(1.5, 1)
    b = a.sin().log() + a**2 * a.cos()

In [10]:
def dfx_tester():
    x = dfx.Dual(2., 1.)
    y = dfx.Dual(3., 2.)

    print(x)

    z1 = x + y
    z2 = x - y
    z3 = x * y
    z4 = x / y
    
    z5 = x.sin()
    z6 = x.cos()
    z7 = x.tan()
    z8 = x.log()
    z9 = x.exp()

    z10 = x**y

    a = dfx.Dual(1.5, 1)
    b = a.sin().log() + a**2 * a.cos()
    print(b)
dfx_tester()

Dual(real=2.0, dual=1.0)


AttributeError: 'dual_autodiff_x.dual.Dual' object has no attribute 'real'

In [15]:
# test.py

import dual_autodiff_x.dual_autodiff_x as dfx

def dfx_tester():
    # Create two Dual objects
    x = dfx.Dual(2.0, 1.0)
    y = dfx.Dual(3.0, 2.0)

    # Print the real and dual parts directly
    print(f"x.real = {x.real}, x.dual = {x.dual}")  # Access attributes directly
    print(f"y.real = {y.real}, y.dual = {y.dual}")  # Access attributes directly

    # Perform operations
    z1 = x + y
    z2 = x - y
    z3 = x * y

    # Print the results
    print(f"z1 = {z1}, z2 = {z2}, z3 = {z3}")

# Run the test
dfx_tester()


AttributeError: 'dual_autodiff_x.dual.Dual' object has no attribute 'real'

In [41]:
N = np.arange(10, 10000, 10)
python = []
cython = []

for n in N:
    x = df.Dual(np.random.rand(), np.random.rand())
    p = timer(lambda:[x*x for _ in range(n)])
    python.append(p)

    y = dfx.Dual(np.random.rand(), np.random.rand())
    c = timer(lambda:[y*y for _ in range(n)])
    cython.append(c)

AttributeError: module 'dual_autodiff_x' has no attribute 'Dual'